# 01 - BigQuery - Table Data Source
Use BigQuery to load and prepare data for machine learning:

### Prerequisites:
-  00 - Environment Setup


---
## Setup

inputs:

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'znguyen'

In [2]:
REGION = 'us-central1'
DATANAME = 'fraud'
NOTEBOOK = '01'

packages:

In [3]:
from google.cloud import bigquery

clients:

In [4]:
bq = bigquery.Client(project = PROJECT_ID)

parameters:

In [5]:
BUCKET = PROJECT_ID

---
## Create Dataset

List BigQuery datasets in the project:

In [6]:
query = f"""
SELECT schema_name
FROM `{PROJECT_ID}.INFORMATION_SCHEMA.SCHEMATA`
"""
bq.query(query = query).to_dataframe()

,schema_name


Create the dataset if missing:

In [7]:
ds = bigquery.Dataset(f"{PROJECT_ID}.{DATANAME}")
ds.location = REGION
ds.labels = {'notebook': f"{NOTEBOOK}"}
ds = bq.create_dataset(dataset = ds, exists_ok = True)

List BigQuery datasets in the project:

In [8]:
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)

fraud


---
## Create Table
- import data from Cloud Storage Bucket
- https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv

In [9]:
destination = bigquery.TableReference.from_string(f"{PROJECT_ID}.{DATANAME}.{DATANAME}")
job_config = bigquery.LoadJobConfig(
    write_disposition = 'WRITE_TRUNCATE',
    source_format = bigquery.SourceFormat.CSV,
    autodetect = True,
    labels = {'notebook':f'{NOTEBOOK}'}
)
job = bq.load_table_from_uri(f"gs://{BUCKET}/{DATANAME}/data/{DATANAME}.csv", destination, job_config = job_config)
job.result()

LoadJob<project=znguyen, location=us-central1, id=4bdaabcd-a111-41ed-b714-8b2c04e803f8>

In [10]:
(job.ended-job.started).total_seconds()

11.76

In [11]:
query = f"""
SELECT *
FROM `{DATANAME}.{DATANAME}`
LIMIT 5
"""
bq.query(query = query).to_dataframe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,282,-0.356466,0.725418,1.971749,0.831343,0.369681,-0.107776,0.751610,-0.120166,-0.420675,...,0.020804,0.424312,-0.015989,0.466754,-0.809962,0.657334,-0.043150,-0.046401,0.0,0
1,380,-1.299837,0.881817,1.452842,-1.293698,-0.025105,-1.170103,0.861610,-0.193934,0.592001,...,-0.272563,-0.360853,0.223911,0.598930,-0.397705,0.637141,0.234872,0.021379,0.0,0
2,403,1.237413,0.512365,0.687746,1.693872,-0.236323,-0.650232,0.118066,-0.230545,-0.808523,...,-0.077543,-0.178220,0.038722,0.471218,0.289249,0.871803,-0.066884,0.012986,0.0,0
3,430,-1.860258,-0.629859,0.966570,0.844632,0.759983,-1.481173,-0.509681,0.540722,-0.733623,...,0.268028,0.125515,-0.225029,0.586664,-0.031598,0.570168,-0.043007,-0.223739,0.0,0
4,711,-0.431349,1.027694,2.670816,2.084787,-0.274567,0.286856,0.152110,0.200872,-0.596505,...,0.001241,0.154170,-0.141533,0.384610,-0.147132,-0.087100,0.101117,0.077944,0.0,0


---
## Review Data
Additional SQL queries could be used to review the data.  This section shows moving the table to a Pandas dataframe for local review in Python:

In [12]:
query = f"""
SELECT *
FROM `{DATANAME}.{DATANAME}`
"""
df = bq.query(query = query).to_dataframe()

In [13]:
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [14]:
df['Class'].value_counts(normalize=True)

0    0.998273
1    0.001727
Name: Class, dtype: float64

---
## Prepare Data for Analysis

Create a prepped version of the data with test/train splits using SQL DDL:

In [15]:
query = f"""
CREATE OR REPLACE TABLE `{DATANAME}.{DATANAME}_prepped` AS
WITH add_id AS(SELECT *, GENERATE_UUID() transaction_id FROM `{DATANAME}.{DATANAME}`)
SELECT *,
    CASE 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 THEN "TRAIN" 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 THEN "VALIDATE"
        ELSE "TEST"
    END AS splits
FROM add_id
"""
job = bq.query(query = query)
job.result()

In [16]:
(job.ended-job.started).total_seconds()

8.521

In [17]:
job.estimated_bytes_processed/1000000 #MB

70.632136

Review the test/train split:

In [18]:
query = f"""
SELECT splits, count(*) as Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
FROM `{DATANAME}.{DATANAME}_prepped`
GROUP BY splits
"""
bq.query(query = query).to_dataframe()

,splits,Count,Percentage
0,VALIDATE,28702,10.077702
1,TEST,28341,9.950949
2,TRAIN,227764,79.971349


Retrieve a subset of the data to a Pandas dataframe:

In [19]:
query = f"""
SELECT * 
FROM `{DATANAME}.{DATANAME}_prepped`
LIMIT 5
"""
data = bq.query(query = query).to_dataframe()

In [20]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,65260,-1.035509,1.055811,2.431036,1.310385,0.383560,0.631163,0.308895,0.164022,-0.447902,...,-0.206376,-0.439529,0.173055,-0.175837,-0.166272,0.113286,0.0,0,cf077393-dd2b-4e47-a29b-26280d7b9d17,TEST
1,66435,1.305572,0.049094,0.305123,-0.065409,-0.604699,-1.203571,0.015596,-0.223136,0.275307,...,0.119527,0.427611,0.156572,0.848560,-0.089827,0.002778,0.0,0,ba585199-b0c8-415e-b115-d71fb6252f06,TEST
2,67884,-1.132191,0.842940,1.596377,-0.272010,-1.546922,-0.698930,-0.867452,0.944616,0.303145,...,0.036098,0.785660,-0.351968,0.922281,-0.381977,-0.126251,0.0,0,7a1c846a-c396-40eb-8268-a70edd322945,TEST
3,73982,-0.904997,1.248734,2.895842,1.573375,0.264535,-0.218413,0.939176,-0.199466,-0.849152,...,-0.184512,0.682007,0.208090,-0.424035,-0.256705,-0.144899,0.0,0,b5947a36-47ef-4285-8ad5-d1754ff92f0b,TEST
4,74468,1.176863,-0.089968,0.762098,0.783237,-0.830532,-0.488663,-0.332575,0.132182,0.509430,...,0.115293,0.506442,0.225750,0.157099,-0.028365,0.004234,0.0,0,0e8419af-4400-4eee-9107-fd352331d009,TEST
